In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from difflib import SequenceMatcher
import re

import warnings
warnings.simplefilter('ignore')

In [44]:
df=pd.read_excel('FINAL_DATASET_777(Q_edu_4).xlsx')

In [5]:
df.drop_duplicates(subset='school_name', keep='first',inplace=True)

In [7]:
df_1=pd.read_excel('output_final.xlsx')

In [5]:
df_1.rename(index=str, columns={'Область': "region", 'Район': 'area','Населенный пункт':'location','Наименование школы':'school', 'Тип организации образования':'type'},inplace=True)

In [9]:
df_1.drop_duplicates(subset='school_name', keep='first',inplace=True)

In [11]:
common_areas=list(set(df_1.area)&set(df.area))

In [14]:
df.area[0]

'Аккольский район'

In [15]:
df_dict=dict(zip(df.school_name,df.area))

In [17]:
keys=list(df_dict.keys())
values=list(df_dict.values())
uniq_val=list(set(values))
dict_df=defaultdict(list)
for val in uniq_val:
    for ind,i in enumerate(values):
        if val==i:
            dict_df[val].append(keys[ind])

In [10]:
del df_dict

In [19]:
df_1dict=dict(zip(df_1.school_name,df_1.area))

In [20]:
keys_=list(df_1dict.keys())
values_=list(df_1dict.values())
uniq_val_=list(set(values_))
dict_df_1=defaultdict(list)
for val in uniq_val_:
    for ind,i in enumerate(values_):
        if val==i:
            dict_df_1[val].append(keys_[ind])

In [21]:
del df_1dict

In [22]:
def proccessing(x):
    
    for i in x:
        ind=x.index(i)
        x[ind]=x[ind].lower()
        x[ind]=re.sub(r'\W+', ' ',x[ind])
        x[ind]=x[ind].replace('кгу','').replace('сш','').replace('гку','')
        x[ind]=x[ind].replace('коммунальное' ,'').replace('государственное учреждение','')
        x[ind]=x[ind].replace('средняя школа', '').replace('имени','').replace('им.','')
        x[ind]=re.sub(' +', ' ',x[ind])
        num=re.findall(r'\d+',x[ind])
        if len(num)>0:
            x[ind]=num[0]
    return x

def get_alt_name(school0,school1,com_ar,proc_func):
    
    result={}
    for area in common_areas:
        sc0=school0[area].copy()
        sc0_=school0[area].copy()
        sc1=school1[area].copy()
        sc1_=school1[area].copy()
        #preproc
        sc0=proc_func(sc0)  
        sc1=proc_func(sc1)
        for s0 in sc0:
            score=[]
            names=[]
            ind0=sc0.index(s0)
            for s1 in sc1:
                ind1=sc1.index(s1)
                f=SequenceMatcher(None, s1,s0).ratio()
                if f>=0.8:
                    score.append(f)
                    names.append(s1)
            if len(score)>0:
                result[sc0_[ind0]]=sc1_[sc1.index(names[np.array(score).argmax()])]
            else:
                result[sc0_[ind0]]=sc0_[ind0]+'_old'
            
    alt_names=list(result.keys())
    my_names=list(result.values())
    result=dict(zip(my_names,alt_names))

    return result      

In [23]:
total_trans=get_alt_name(dict_df_1,dict_df,common_areas,proccessing)

In [24]:
common_schools_0={}
common_schools_1={}
for name in total_trans:
    if not name.endswith('_old'):
        common_schools_0[name]=total_trans[name]
        common_schools_1[total_trans[name]]=name

In [25]:
tt=len(total_trans)
cs=len(common_schools_0)
print('Переведено: '+str(cs)+' из '+str(tt))

Переведено: 1318 из 2053


In [29]:
df['alt_names']=df.school_name.map(common_schools_0)

In [30]:
df.drop(df[df.alt_names!=df.alt_names].index, inplace=True)

In [31]:
df_1['alt_names']=df_1.school_name

In [32]:
merged_df=pd.merge(df,df_1,on='alt_names',how='left')

In [57]:
# writing 

writer = pd.ExcelWriter(r"final_dataset_w_q_edu_somnitelno.xlsx", engine='xlsxwriter', options={'strings_to_urls': False})
result.to_excel(writer)
writer.close()

In [43]:
df1 = pd.read_excel("final_dataset_w_q_edu.xlsx")

In [55]:
result = pd.merge(df, df1, on='ID', how = 'left')